In [54]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [55]:
team_box1617 = pd.read_csv("./2016-17_teamBoxScore.csv", sep=',', dtype = str)
player_box1617 = pd.read_csv("./2016-17_playerBoxScore.csv", sep=',', dtype = str)
official_box1617 = pd.read_csv("./2016-17_officialBoxScore.csv", sep=',', dtype = str)
standings1617 = pd.read_csv("./2016-17_standings.csv", sep=',', dtype = str)
team_box1718 = pd.read_csv("./2017-18_teamBoxScore.csv", sep=',', dtype = str)
player_box1718 = pd.read_csv("./2017-18_playerBoxScore.csv", sep=',', dtype = str)
official_box1718 = pd.read_csv("./2017-18_officialBoxScore.csv", sep=',', dtype = str)
standings1718 = pd.read_csv("./2017-18_standings.csv", sep=',', dtype = str)

In [56]:
names = ["Team", "W", "L", 'Pct', 'GB', 'Home', 'Road', "Conf", "Div", "Streak", "L10", "?"]
team_statistics = pd.read_csv("./scraping_attempt2.csv", sep = ',', dtype = str, names = names)
del team_statistics['?']
team_statistics.drop(0, inplace=True)
team_statistics.drop(16, inplace=True)

In [57]:
team_statistics

,Team,W,L,Pct,GB,Home,Road,Conf,Div,Streak,L10
1,z-Toronto,59,23,0.720,-,34-7,25-16,40-12,12-4,L1,6-4
2,x-Boston,55,27,0.671,4,27-14,28-13,33-19,12-4,W1,6-4
3,x-Philadelphia,52,30,0.634,7,30-11,22-19,34-18,9-7,W16,10-0
4,y-Cleveland,50,32,0.610,9,29-12,21-20,35-17,11-5,L1,7-3
5,x-Indiana,48,34,0.585,11,27-14,21-20,32-20,10-6,L1,7-3
6,y-Miami,44,38,0.537,15,26-15,18-23,31-21,11-5,W1,5-5
7,x-Milwaukee,44,38,0.537,15,25-16,19-22,27-25,6-10,L1,6-4
8,x-Washington,43,39,0.524,16,23-18,20-21,28-24,8-8,L1,3-7
9,o-Detroit,39,43,0.476,20,25-16,14-27,24-28,9-7,W1,7-3
10,o-Charlotte,36,46,0.439,23,21-20,15-26,22-30,11-5,W1,5-5
